In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcsetup.all_backends

import pyExSi

import LadiskDAQ

In [3]:
acq = LadiskDAQ.NIAcquisition('TestInputTask')
acq.set_trigger(10, 0, duration=10)

In [4]:
fs = 51200 # output sample rate
time_array = np.arange(fs) / fs  

signal1 = pyExSi.sine_sweep(time_array, freq_start=100, freq_stop=2000)
#signal2 = 0.1*np.sin(time_array*2*np.pi*50-0.4*np.pi) + 0.01*np.cos(time_array*2*np.pi*100-np.pi)
signal2 = np.sin(time_array*2*np.pi*500)
signal2 = - signal1 * 2

signal = np.array([signal1, signal2])


#plt.figure()
#plt.plot(time_array, signal1)

signal.shape

(2, 51200)

In [5]:
gen = LadiskDAQ.NIGenerator('TestOutputTask', signal)

In [6]:
def fun1(self, data):
    return data**2 + 10

def fun2(self, data):
    return data**2 - 2

def fun3(self, data):
    X = np.fft.rfft(data)*2/len(data)
    freq = np.fft.rfftfreq(len(data), d=1/self.acquisition.sample_rate)
    return np.array([freq, np.abs(X)]).T

def fun4(self, data):
    x, y = data.T
    x, y = x**2+100, y**3-30
    return np.array([x, y]).T

plot_layout={
    (0, 0): [1],
    (0, 1): [0],
    (1, 0): [1, "fft"],
    (1, 1): [0, "fft"],
    (2, 0): [(0, 1), "frf_amp"],
    (2, 1): [(0, 1), "frf_phase"]
}

#plot_layout = 'default'

ldaq = LadiskDAQ.LDAQ(acq, gen)
ldaq.configure(plot_layout=plot_layout, max_time=1, autoclose=True, nth_point=1)

In [ ]:
ldaq.run()

Press "q" to stop measurement.
	Waiting for trigger...

In [9]:
plt.figure()
plt.plot(acq.data[:, 0][:len(acq.data)//3])
# plt.plot(signal.T)

['Voltage', 'Voltage_0']